In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [3]:
mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils

In [3]:
#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))

In [4]:
# extract keypoints of the features in image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

In [6]:
os.path.join("Sign_Images")

'Sign_Images'

In [3]:
#path for flatten datas
dataPath = os.path.join('Sign_Data')

#path for images
imgPath= os.path.join('Sign_Image')

signs=np.array(["Hello","Thank You", "Hungry", "Food", "Hospital", "Washroom"])

#30 video of each sign
numSequences=30

#length of each video
sequenceLength=30

In [7]:
for sign in signs:
    for sequence in range(numSequences):
            try:
                os.makedirs(os.path.join(dataPath,sign,str(sequence)))
                os.makedirs(os.path.join(imgPath,sign,str(sequence)))
            except:
                pass
# if not os.path.exists("Sign_Data"):
#     os.mkdir("Sign_Data")
# else:
#     print("already exist")


In [5]:
os.path??

In [9]:
# while True:
#     inpt=input(print("Select which data are you collecting:\n0: Hello \n1: Thank you"))
#     if inpt == '0' or inpt == '1':
#         choice=signs[int(inpt)]
#         break
#     else:
#         print("Invalid input, please select again!!")
# choice   

In [10]:
vid=cv2.VideoCapture(0)
with mpHolistic.Holistic() as holistic:

    #taking label input 
     while True:
        blankImg = np.zeros(shape=[512, 512, 3], dtype=np.uint8)
        cv2.putText(blankImg,"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"0: Hello , 1: Thank You", (10,100),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"2: Hungry , 3: Food", (10,150),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"4: Hospital , 5: Washroom", (10,200),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(blankImg,"'ESC' to escape", (10,250),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('Select label',blankImg)
        inpt=cv2.waitKey(0)
        if inpt == 48 or inpt == 49 or inpt == 50 or inpt == 51 or inpt == 52 or inpt == 53:
            choice=signs[inpt-48]
            cv2.destroyWindow('Select label')
            break
        else:
            if inpt==27:
                break
            
    #checking camera is opened or not and taking data    
     while vid.isOpened() and inpt!=27 :
        for sequence in range(numSequences):
            for frameNum in range(sequenceLength+1):
                #checks for user input to close the windows                
                key=cv2.waitKey(1)                  
                success,img=vid.read()
                
                #checking if data is accessed or not from camera
                if not success:
                    break  
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                # img.flags.writeable=False
                results=holistic.process(img) 
                # img.flags.writeable=True     
                img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
                
                #draws landmarks
                drawLandmarks()
                
                #show feed for collecting datas and delays for 2 sec
                if frameNum == 0: 
                    cv2.putText(img, "Press 'ESC' to escape", (10,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.putText(img, 'press any key to conitnue', (10,60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0, 0), 1, cv2.LINE_AA)
                    cv2.imshow('Collecting Datas', img)
                    key= cv2.waitKey(0)
                    cv2.waitKey(1000)
                    
                #starts collecting datas    
                else: 
                    cv2.putText(img, f"Collecting Data for '{choice}' Video Number {sequence}", (15,20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA) 
                    cv2.imshow('Collecting Datas', img)
                    keypoints=extractKeypoints()  
                    
                    #saving flatten array to datapath
                    np.save(os.path.join(dataPath,choice,str(sequence),str(frameNum-1)),keypoints)
                    
                    #saving image to image path
                    jpgPath=os.path.join(imgPath,choice,str(sequence),str(frameNum-1))
                    cv2.imwrite(f"{jpgPath}.jpg",img)
                    key=2 #giving default value for key to avoid esc while taking data
                
                if key == 27 : #press esc to close the window
                    break     

            if key == 27 : #press esc to close the window
                break        
        if key == 27 : #press esc to close the window
            break             
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()            

# Preprocess Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(signs)}

In [ ]:
label_map

{'Hello': 0,
 'Thank You': 1,
 'Hungry': 2,
 'Food': 3,
 'Hospital': 4,
 'Washroom': 5}

In [11]:
label_map[sign]

5

In [10]:
sequences, labels = [], []
for sign in signs:
    for sequence in range(numSequences):
        window = []
        for frameNum in range(sequenceLength):
            res = np.load(os.path.join(dataPath, sign, str(sequence), "{}.npy".format(frameNum)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[sign])

In [ ]:
labels

In [13]:
np.array(labels).shape

(180,)

In [ ]:
np.array(sequences).shape

(180, 30, 1662)

In [17]:
arr=np.array([[[1,2],[3,4],[5,6]],
                 [[1,2],[3,4],[5,6]]])

In [18]:
arr.shape

(2, 3, 2)

In [11]:
X = np.array(sequences)

In [ ]:
X.shape

(180, 30, 1662)

In [12]:
y = to_categorical(labels).astype(int)

In [ ]:
y.shape

(180, 6)

In [ ]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [25]:
y_test.shape

(9, 6)

# Save test datas and signs

In [14]:
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
try:
    os.makedirs(os.path.join(testDatasPath))
    os.makedirs(os.path.join(signsPath))
except:
    pass
np.save(os.path.join(testDatasPath,'x_test.npy'),X_test)
np.save(os.path.join(testDatasPath,'y_test.npy'),y_test)
np.save(os.path.join(signsPath,'signs.npy'),signs)

# Build and Train LSTM Neural Network

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

Build and Train model

In [16]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))


In [7]:
signs.shape[0]

6

In [ ]:
res = [.7, 0.2, 0.1]

In [ ]:
signs[np.argmax(res)]

'Hello'

In [17]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [19]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
6/6 [==============================] - 9s 363ms/step - loss: 20.6326 - categorical_accuracy: 0.1637
Epoch 2/2000
6/6 [==============================] - 2s 372ms/step - loss: 71.8506 - categorical_accuracy: 0.1988
Epoch 3/2000
6/6 [==============================] - 2s 358ms/step - loss: 108.5349 - categorical_accuracy: 0.2164
Epoch 4/2000
6/6 [==============================] - 2s 355ms/step - loss: 163.8445 - categorical_accuracy: 0.1813
Epoch 5/2000
6/6 [==============================] - 2s 349ms/step - loss: 107.1591 - categorical_accuracy: 0.1228
Epoch 6/2000
6/6 [==============================] - 2s 348ms/step - loss: 57.7937 - categorical_accuracy: 0.1170
Epoch 7/2000
6/6 [==============================] - 2s 359ms/step - loss: 59.4696 - categorical_accuracy: 0.1287
Epoch 8/2000
6/6 [==============================] - 2s 329ms/step - loss: 46.6790 - categorical_accuracy: 0.1871
Epoch 9/2000
6/6 [==============================] - 2s 343ms/step - loss: 31.3910 - categoric

6/6 [==============================] - 2s 315ms/step - loss: 1.5970 - categorical_accuracy: 0.3158
Epoch 146/2000
6/6 [==============================] - 2s 322ms/step - loss: 1.4408 - categorical_accuracy: 0.3977
Epoch 147/2000
6/6 [==============================] - 2s 325ms/step - loss: 1.4436 - categorical_accuracy: 0.4327
Epoch 148/2000
6/6 [==============================] - 2s 338ms/step - loss: 1.4835 - categorical_accuracy: 0.3918
Epoch 149/2000
6/6 [==============================] - 2s 310ms/step - loss: 1.6459 - categorical_accuracy: 0.3567
Epoch 150/2000
6/6 [==============================] - 2s 253ms/step - loss: 1.7554 - categorical_accuracy: 0.4269
Epoch 151/2000
6/6 [==============================] - 2s 273ms/step - loss: 1.7051 - categorical_accuracy: 0.4152
Epoch 152/2000
6/6 [==============================] - 1s 232ms/step - loss: 1.2801 - categorical_accuracy: 0.4912
Epoch 153/2000
6/6 [==============================] - 2s 259ms/step - loss: 1.2180 - categorical_accura

Epoch 217/2000
6/6 [==============================] - 2s 320ms/step - loss: 1.3206 - categorical_accuracy: 0.5380
Epoch 218/2000
6/6 [==============================] - 2s 313ms/step - loss: 1.2036 - categorical_accuracy: 0.5497
Epoch 219/2000
6/6 [==============================] - 2s 326ms/step - loss: 1.0859 - categorical_accuracy: 0.5848
Epoch 220/2000
6/6 [==============================] - 2s 326ms/step - loss: 0.7821 - categorical_accuracy: 0.6433
Epoch 221/2000
6/6 [==============================] - 2s 318ms/step - loss: 0.9497 - categorical_accuracy: 0.6023
Epoch 222/2000
6/6 [==============================] - 2s 335ms/step - loss: 1.0054 - categorical_accuracy: 0.5731
Epoch 223/2000
6/6 [==============================] - 2s 330ms/step - loss: 0.9491 - categorical_accuracy: 0.5906
Epoch 224/2000
6/6 [==============================] - 2s 323ms/step - loss: 0.6781 - categorical_accuracy: 0.7193
Epoch 225/2000
6/6 [==============================] - 2s 323ms/step - loss: 0.6490 - cat

Epoch 360/2000
6/6 [==============================] - 2s 311ms/step - loss: 0.7522 - categorical_accuracy: 0.7602
Epoch 361/2000
6/6 [==============================] - 2s 314ms/step - loss: 0.4028 - categorical_accuracy: 0.7836
Epoch 362/2000
6/6 [==============================] - 2s 342ms/step - loss: 0.3322 - categorical_accuracy: 0.8830
Epoch 363/2000
6/6 [==============================] - 2s 344ms/step - loss: 0.2647 - categorical_accuracy: 0.9240
Epoch 364/2000
6/6 [==============================] - 2s 333ms/step - loss: 0.2182 - categorical_accuracy: 0.9357
Epoch 365/2000
6/6 [==============================] - 2s 333ms/step - loss: 0.1956 - categorical_accuracy: 0.9357
Epoch 366/2000
6/6 [==============================] - 2s 363ms/step - loss: 0.3211 - categorical_accuracy: 0.8889
Epoch 367/2000
6/6 [==============================] - 2s 348ms/step - loss: 0.2450 - categorical_accuracy: 0.8713
Epoch 368/2000
6/6 [==============================] - 2s 346ms/step - loss: 0.7607 - cat

Epoch 503/2000
6/6 [==============================] - 2s 323ms/step - loss: 0.5070 - categorical_accuracy: 0.8596
Epoch 504/2000
6/6 [==============================] - 2s 315ms/step - loss: 1.3603 - categorical_accuracy: 0.6959
Epoch 505/2000
6/6 [==============================] - 2s 308ms/step - loss: 0.5868 - categorical_accuracy: 0.8304
Epoch 506/2000
6/6 [==============================] - 2s 339ms/step - loss: 0.3352 - categorical_accuracy: 0.9006
Epoch 507/2000
6/6 [==============================] - 2s 324ms/step - loss: 0.3147 - categorical_accuracy: 0.8830
Epoch 508/2000
6/6 [==============================] - 2s 331ms/step - loss: 0.5217 - categorical_accuracy: 0.8363
Epoch 509/2000
6/6 [==============================] - 2s 327ms/step - loss: 1.0623 - categorical_accuracy: 0.6784
Epoch 510/2000
6/6 [==============================] - 2s 318ms/step - loss: 0.9534 - categorical_accuracy: 0.7018
Epoch 511/2000
6/6 [==============================] - 2s 316ms/step - loss: 0.5544 - cat

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-tr

# Make Prediction

In [31]:
#load signs, x_test, y_test
testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))

In [32]:
res = model.predict(x_test)

In [41]:
signs[np.argmax(res[4])]

'Hospital'

In [42]:
signs[np.argmax(y_test[4])]

'Hungry'

In [ ]:
res.shape

(13, 6)

In [ ]:
y_test

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
res

In [43]:
np.argmax(res[0])

4

In [44]:
np.argmax(y_test[0])

4

# Save Model

In [46]:
model.save('SLR.h5')

# Load Model

NameError: name 'createModel' is not defined

In [4]:
model.load_weights('SLR.h5')

NameError: name 'model' is not defined

# Evaluation using confusion matrix

In [47]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix, accuracy_score

In [48]:
true_labels=[]
for label in y_test:
    index=np.argmax(label)
    true_labels.append(signs[index])

In [49]:
true_labels

['Hospital',
 'Hungry',
 'Hospital',
 'Hello',
 'Hungry',
 'Thank You',
 'Hello',
 'Food',
 'Food']

In [50]:
y_predicted = model.predict(x_test)

In [51]:
predicted_labels=[]
for label in y_predicted:
    index=np.argmax(label)
    predicted_labels.append(signs[index])

In [52]:
predicted_labels

['Hospital',
 'Hungry',
 'Thank You',
 'Hello',
 'Hospital',
 'Thank You',
 'Hello',
 'Thank You',
 'Food']

In [53]:
y_true = np.argmax(y_test, axis=1).tolist()

In [54]:
y_predicted=np.argmax(y_predicted,axis=1).tolist()

In [55]:
y_true

[4, 2, 4, 0, 2, 1, 0, 3, 3]

In [ ]:
y_test

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [ ]:
y_test.shape

(13, 6)

In [ ]:
y_predicted

[4, 5, 3, 3, 1, 4, 0, 0, 0, 1, 0, 2, 0]

In [ ]:
signs

array(['Hello', 'Thank You', 'Hungry', 'Food', 'Hospital', 'Washroom'],
      dtype='<U9')

In [56]:
multilabel_confusion_matrix(true_labels, predicted_labels,labels=signs)

array([[[7, 0],
        [0, 2]],

       [[6, 2],
        [0, 1]],

       [[7, 0],
        [1, 1]],

       [[7, 0],
        [1, 1]],

       [[6, 1],
        [1, 1]],

       [[9, 0],
        [0, 0]]], dtype=int64)

In [57]:
confusion_matrix(y_true, y_predicted)

array([[2, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1]], dtype=int64)

In [58]:
accuracy_score(y_true,y_predicted)

0.6666666666666666

In [ ]:
y_test.tolist()

[[0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0]]

In [59]:
list(zip(true_labels, predicted_labels))

[('Hospital', 'Hospital'),
 ('Hungry', 'Hungry'),
 ('Hospital', 'Thank You'),
 ('Hello', 'Hello'),
 ('Hungry', 'Hospital'),
 ('Thank You', 'Thank You'),
 ('Hello', 'Hello'),
 ('Food', 'Thank You'),
 ('Food', 'Food')]

In [5]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import os,cv2
import mediapipe as mp
import time

mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils
vid=cv2.VideoCapture(0)
sequenceLength=30

#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))


#extract keypoints from image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

#load signs, x_test, y_test
# testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
# x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
# y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))



#load model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))  # 128
model.add(LSTM(128, return_sequences=True, activation='relu'))  # 64
model.add(LSTM(64, return_sequences=False, activation='relu'))  # 64
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))
model.load_weights('SLR.h5')
print(model.summary())
# real time testing
sequence=[]
sentence=[]
predictions = []
threshold=0.5
prediction_duration=[]
start_time=0.0
end_time=0.0
start_timer=True

dummy_arr=np.zeros([10,1662],dtype=float)

dummy_lst=[]

for e in dummy_arr:
    dummy_lst.append(list(e))

# ['Hello' 'Thank You' 'Hungry' 'Food' 'Hospital' 'Washroom']
def get_threshold(index):
    switcher = {
        0: 1.0, # for Hello
        1: 0.5,  # for Thank You
        2: 0.5,  # for Hungry
        3: 0.02,  # for Food
        4: 0.3,  # for Hospital
        5: 0.3   # for Washroom
    } 
    return switcher.get(index)


with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                     
    #checking camera is opened or not and taking data    
    while vid.isOpened():
        if start_timer:
            start_time=time.time()
        #checks for user input to close the windows                
        key=cv2.waitKey(1)                  
        success,img=vid.read()
        
        #checking if data is accessed or not from camera
        if not success:
            break  

        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        # img.flags.writeable=False
        results=holistic.process(img) 
        # img.flags.writeable=True     
        img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

        #draws landmarks
        drawLandmarks()

        #starts collecting datas    
        keypoints=extractKeypoints() 

        #appending to sequence list 
        sequence.append(keypoints)
        print(len(sequence))
        sequence = sequence[-30:]
        if len(sequence)==30:
            #prediction   
            # print(np.array(sequence).shape)
            # print(np.expand_dims(sequence, axis=0).shape)
            # temp_sequence=sequence
            # temp_sequence.append(dummy_lst)
            res=model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
            predictions.append(np.argmax(res))
            
            #res=[1.0 if x>1 else x for x in res ] 
            # print(res)
            print(signs[np.argmax(res)])
            # threshold_val=get_threshold(np.argmax(res))
            if np.unique(predictions[-20:])[0]==np.argmax(res): 
                
                print(predictions[-20:])
                if res[np.argmax(res)] > threshold:
                     

                    if len(sentence) > 0: 
                        if signs[np.argmax(res)] != sentence[-1]:
                            sentence.append(signs[np.argmax(res)])
                            start_timer=True
                            end_time=time.time()
                            prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])

                        else:
                            start_timer=False    
                            
                    else:
                        end_time=time.time()
                        prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])
                        sentence.append(signs[np.argmax(res)])
                        start_timer=True
            else:
                start_timer=False
        if len(sentence) > 5: 
            sentence =[]
            # cv2.putText('Collecting Datas',"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA) 
            # cv2.imshow('Collecting Datas', img)
        
        cv2.rectangle(img, (0,0), (640, 40), (245, 117, 16), -1)     
        cv2.putText(img,' '.join(sentence), (10,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA) 
        cv2.imshow('SLR',img)
    
            
        if key == 27 : #press esc to close the window
            break   
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()

print(prediction_duration)

FileNotFoundError: [Errno 2] No such file or directory: 'Signs\\signs.npy'

In [1]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import os,cv2
import mediapipe as mp
import time

mpHolistic=mp.solutions.holistic
mpDraw=mp.solutions.drawing_utils
vid=cv2.VideoCapture(0)
sequenceLength=30

#draw landmarks in image
def drawLandmarks():
    mpDraw.draw_landmarks(img,results.face_landmarks,mpHolistic.FACEMESH_CONTOURS,mpDraw.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 256, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.pose_landmarks,mpHolistic.POSE_CONNECTIONS,mpDraw.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.left_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(121, 22, 90), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(255, 100, 112), thickness=2, circle_radius=2))
    mpDraw.draw_landmarks(img,results.right_hand_landmarks,mpHolistic.HAND_CONNECTIONS,mpDraw.DrawingSpec(color=(250, 80, 10), thickness=1, circle_radius=1),mpDraw.DrawingSpec(color=(200, 50, 250), thickness=2, circle_radius=2))


#extract keypoints from image
def extractKeypoints():
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    leftHand=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten()if results.left_hand_landmarks else np.zeros(21*3)
    rightHand=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten()if results.right_hand_landmarks else np.zeros(21*3)
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([face,leftHand,rightHand,pose])

#load signs, x_test, y_test
# testDatasPath=os.path.join("Test_Datas")
signsPath=os.path.join("Signs")
# x_test=np.load(os.path.join(testDatasPath,'x_test.npy'))
# y_test=np.load(os.path.join(testDatasPath,'y_test.npy'))
signs=np.load(os.path.join(signsPath,'signs.npy'))



#load model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))  # 128
model.add(LSTM(128, return_sequences=True, activation='relu'))  # 64
model.add(LSTM(64, return_sequences=False, activation='relu'))  # 64
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))
model.load_weights('SLR.h5')
print(model.summary())
# real time testing
sequence=[]
sentence=[]
predictions = []
threshold=0.5
prediction_duration=[]
start_time=0.0
end_time=0.0
start_timer=True

dummy_arr=np.zeros([10,1662],dtype=float)

dummy_lst=[]

for e in dummy_arr:
    dummy_lst.append(list(e))

# ['Hello' 'Thank You' 'Hungry' 'Food' 'Hospital' 'Washroom']
def get_threshold(index):
    switcher = {
        0: 1.0, # for Hello
        1: 0.5,  # for Thank You
        2: 0.5,  # for Hungry
        3: 0.02,  # for Food
        4: 0.3,  # for Hospital
        5: 0.3   # for Washroom
    } 
    return switcher.get(index)


with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                     
    #checking camera is opened or not and taking data    
    while vid.isOpened():
        if start_timer:
            start_time=time.time()
        #checks for user input to close the windows                
        key=cv2.waitKey(1)                  
        success,img=vid.read()
        
        #checking if data is accessed or not from camera
        if not success:
            break  

        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        # img.flags.writeable=False
        results=holistic.process(img) 
        # img.flags.writeable=True     
        img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

        #draws landmarks
        drawLandmarks()

        #starts collecting datas    
        keypoints=extractKeypoints() 

        #appending to sequence list 
        sequence.append(keypoints)
        print(len(sequence))
        sequence = sequence[-30:]
        if len(sequence)==30:
            #prediction   
            # print(np.array(sequence).shape)
            # print(np.expand_dims(sequence, axis=0).shape)
            # temp_sequence=sequence
            # temp_sequence.append(dummy_lst)
            res=model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
            predictions.append(np.argmax(res))
            
            #res=[1.0 if x>1 else x for x in res ] 
            # print(res)
            print(signs[np.argmax(res)])
            # threshold_val=get_threshold(np.argmax(res))
            if np.unique(predictions[-20:])[0]==np.argmax(res): 
                
                print(predictions[-20:])
                if res[np.argmax(res)] > threshold:
                     

                    if len(sentence) > 0: 
                        if signs[np.argmax(res)] != sentence[-1]:
                            sentence.append(signs[np.argmax(res)])
                            start_timer=True
                            end_time=time.time()
                            prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])

                        else:
                            start_timer=False    
                            
                    else:
                        end_time=time.time()
                        prediction_duration.append([signs[predictions[-1]],round(end_time-start_time,2)])
                        sentence.append(signs[np.argmax(res)])
                        start_timer=True
            else:
                start_timer=False
        if len(sentence) > 5: 
            sentence =[]
            # cv2.putText('Collecting Datas',"Select label: ", (10,50),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA) 
            # cv2.imshow('Collecting Datas', img)
        
        cv2.rectangle(img, (0,0), (640, 40), (245, 117, 16), -1)     
        cv2.putText(img,' '.join(sentence), (10,25),cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA) 
        cv2.imshow('SLR',img)
    
            
        if key == 27 : #press esc to close the window
            break   
            
#releasing the port            
vid.release() 

#destroying all opened windows using opencv
cv2.destroyAllWindows()

print(prediction_duration)

FileNotFoundError: [Errno 2] No such file or directory: 'Signs\\signs.npy'